In [28]:
import json

In [29]:
with open('mascqa-eval.json','r') as f:
    questions_file = json.load(f)
f.close()

In [30]:
import pandas as pd
import os

df = pd.DataFrame(questions_file)

In [31]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

def ask_questions(questions):
    """
    Ask a list of questions to an LLM model like gpt-3.5-turbo and get the answers in a list format.

    Parameters:
    questions (list): A list of questions to ask the model.

    Returns:
    list: A list of answers provided by the model.
    """
    
    prompt = "Solve the following question. Write the correct answer inside a list at the end."
    
    answers = []
    total_token = []
    for question in questions:
        response = client.chat.completions.create(
            model="gpt-4o",# gpt-4o-2024-05-13 
            temperature=0,
            messages=[
                {"role": "assistant", "content":prompt},
                {"role": "user", "content": question}
            ]
        )
        
        # Extract the response from the model
        model_response = response.choices[0].message.content
        total_token.append(response.usage)
        answers.append(model_response)

    return answers, total_token, response.model


# Example usage:
questions = [
    "What is the capital of France?",
    "What is 2 + 2?",
    "Who wrote 'To Kill a Mockingbird'?"
]

answers, total_token, model = ask_questions(questions)

In [32]:
total_token=[]
for i in questions_file.keys():
    newpath = f'./answer_gpt_4_o/{i}'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    questions = df[f"{i}"]["questions"]
    answers, Total_Token, model = ask_questions(questions)
    total_token.append(Total_Token)
    for j in range(len(answers)):
        a = df[f"{i}"]["qids"][j]
        with open(f"./{newpath}/{a}.txt",'w') as file:
            file.write(answers[j])
    

In [33]:
with open(f"./answer_gpt_4_o/report{model}.txt",'w') as file:
    file.write(str(total_token))

In [34]:
input_token=0
output_token=0

for i in range(len(total_token)):
    for j in range(len(total_token[i])):
        input_token += total_token[i][j].prompt_tokens
        output_token += total_token[i][j].completion_tokens

In [35]:
print("Input token =",input_token, ", Output Token =",output_token)

Input token = 76273 , Output Token = 262636
